# Multi-Agent Research System

## Overview

This notebook demonstrates a multi-agent research system that uses OpenAI's agents to perform research along the lines of Anthropicls blog post [How we built our multi-agent research system](https://www.anthropic.com/engineering/built-multi-agent-research-system).

In [14]:
# import libraries
from dotenv import load_dotenv
from pydantic import BaseModel, Field
from agents import Agent, Runner, trace, function_tool

In [15]:
# load environment variables
load_dotenv(override=True)

True

## Lead Researcher Agent

The Lead Researcher Agent is responsible for:

- Creating a research plan and saving it to memory
- Creating N specialized sub-agents with specific research tasks
- Synthesizing the results from the sub-agents into a final report format
- Evaluate if more research is needed to answer the research question
- Passing the final report to the Citations Agent

In [16]:
# create the data model for the output


class WebSearchItem(BaseModel):
    """A single item to be searched on the web."""

    query: str = Field(description="The query to search the web for")
    reason: str = Field(description="The reason for the query")


class WebSearchPlan(BaseModel):
    """A plan for searching the web."""

    items: list[WebSearchItem]


class ResearchPlan(BaseModel):
    """A plan for the research."""

    research_plan: str = Field(
        description="The research plan, including the steps to be taken to achieve the goal"
    )
    number_of_sub_agents: int = Field(
        description="The number of sub-agents to be spawned for the research"
    )
    web_search_plan: WebSearchPlan = Field(description="The plan for searching the web")


class ResearchOutput(BaseModel):
    """The output of the research system."""

    goal: str = Field(description="The goal of the research")
    research_plan: ResearchPlan = Field(description="The research plan")
    report_summary: str = Field(description="A summary of the report to be generated")
    final_report: str = Field(
        description="The final report, including the results of each subagent synthesized accoring to the research plan"
    )

In [19]:
# create the tools for the lead researcher agent

goal = """What are the companies in the United States working on A.I. agents in 2025? 
Make a list of at least 100. For each company, include the name, website, product, 
description of what they do, type of agents they build, and their vertical/industry."""

planner_instructions = f"""
You are a research planner agent. Your task is to generate a comprehensive research plan to achieve the following goal:

**Goal:** {goal}

Your plan must include:

1. **Search Strategy**  
   - Describe the overall approach to answering the goal, including which sources, tools, or databases to use.
   - Identify key search terms, sub-questions, or specific aspects to investigate.

2. **What to Search For**  
   - List the types of information, evidence, or resources needed, with a brief explanation for each.

3. **How Many to Search For**
   - You must decide how many items to search for, and why that number is sufficient.
   - For each type of information, specify the search terms and the reason for that search term.

4. **Step-by-Step Plan**  
   - Outline the sequence of steps to follow.

Please format your output using this template:

---
### 1. Search Strategy
- Approach: 
- Key Terms & Sub-questions: 

### 2. What to Search For
- Resource 1: (what and why)
- Resource 2: (what and why)
- ...

### 3. How Many to Search For
- Item 1: (search term and reason)
- Item 2: (search term and reason)
- ...

### 4. Step-by-Step Plan
1. (description)
2. (description)
3. ...

---

Do not perform the research—only provide the plan.
"""

planner_agent = Agent(
    name="Planner Agent",
    instructions=planner_instructions,
    output_type=ResearchPlan,
    model="o3",
)

In [20]:
with trace("Multi-Agent Research System"):
    result = await Runner.run(planner_agent, goal)

print(result.final_output)

research_plan='---\n### 1. Search Strategy\n- Approach:\n  Combine broad web search, startup/VC databases, technology news outlets, social media, and conference/summit speaker lists to compile an over-sized long-list (≈150+) of U.S. companies declaring work on “AI agents” or closely related terms (autonomous agents, AI copilots, agentic workflow, LLM agents, multi-agent systems) in 2024-2025. Then filter to at least 100 unique firms, extract required metadata from official websites, press kits, Crunchbase, LinkedIn, product docs, and reviews.  Use four specialized sub-agents: (1) Discovery, (2) Verification & Fact-gathering, (3) Data Structuring, (4) QA & Gap-fill.\n- Key Terms & Sub-questions:\n  • Core terms: “AI agent startup”, “autonomous agent platform”, “LLM agents company”, “AI copilot SaaS”, “multi-agent systems company”, “agentic AI”.\n  • Vertical variants: add keywords like customer service, devtools, cybersecurity, healthcare, finance, marketing, e-commerce, robotics, legal